In [ ]:
include("./trajopt/utils.jl")
include("./trajopt/dynamics.jl")
include("./trajopt/constraint.jl")
include("./trajopt/costfunction.jl")
include("./trajopt/scaling.jl")
include("./trajopt/scp.jl")
include("./trajopt/discretize.jl")

In [ ]:
ix = 3
iu = 2
N = 15
dynamics = Unicycle()

In [ ]:
function get_H_obs(rx,ry)
    return diagm([1/rx,1/ry])
end
c_list = []
H_list = []
c1 = [1,2]
H1 = get_H_obs(0.75,1.5)
push!(c_list,c1)
push!(H_list,H1)
c2 = [4,3]
H2 = get_H_obs(0.75,1.5)
push!(c_list,c2)
push!(H_list,H2)

In [ ]:
vmax = 2
vmin = 0
wmax = 1
wmin = -1
Ac = [1 0;0 1;-1 0;0 -1];
bc = [vmax;wmax;-vmin;-wmin];
input_const = InputLinear(Ac,bc)
obstacle_const1 = Obstacle(H_list[1],c_list[1])
obstacle_const2 = Obstacle(H_list[2],c_list[2])
list_const = [input_const,obstacle_const1,obstacle_const2]

In [ ]:
xi = [0;0;0];
xf = [5;5;0];

In [ ]:
x0 = zeros(ix,N+1);
u0 = 0.1*ones(iu,N+1);
tf0 = 5.0;
dt0 = tf0/N*ones(N);
for i = 1:N+1
    x0[:,i] = (N-i+1)/N*xi+(i-1)/N*xf;
end

In [ ]:
p2 = Plots.plot(; size=(500,500))
plot!(x0[1,:],x0[2,:],c=:deepskyblue3,linestyle=:dash,linewidth=1.5,label=nothing)
scatter!(x0[1,:],x0[2,:],c=:deepskyblue3,linewidth=1.5,label=nothing)
for (idx,(ce, H)) in enumerate(zip(c_list, H_list))
    label = nothing
    if idx == 1
        label = "obstacle"
    end
    plot_ellipse(p2,inv(H)*inv(H),ce,"red3",label=label)
end
annotate!(0.2, 0.5, text("Initial", :black, :center))
annotate!(4.8, 4.5, text("Final", :black, :center))
display(p2)

In [ ]:
xmin = [0;0;0];
xmax = [5;5;pi];
umin = [0;0];
umax = [2;1];
min_dt = 0.1;
# max_dt = 0.5;
max_dt = 2;
scaler = Scaling(xmin, xmax, umin, umax, tf0, min_dt,max_dt)
@assert max_dt * N >= tf0 

In [ ]:
max_iter = 30;
w_tf = 0.0;
w_c = 1e-1;
w_rate = 1e-3;
w_param = 0.0;
w_vc = 1e2;
w_tr::Float64 = 5*1e-1;
tol_vc = 1e-6;
tol_tr = 1e-4;
tol_dyn = 1e-1;
tr_norm = "quad";
verbosity = true;

In [ ]:
ptr = PTR(N,tf0,max_iter,dynamics,list_const,scaler,
    w_tf,w_c,w_rate,w_param,w_vc,w_tr,
    tol_vc,tol_tr,tol_dyn,
    tr_norm,verbosity)

In [ ]:
run(ptr,x0,u0,dt0,xi,xf,"Gurobi")

In [ ]:
p2 = Plots.plot(; size=(500,500))
plot!(ptr.solution.xprop[1,:],ptr.solution.xprop[2,:],c=:deepskyblue3,linestyle=:dash,linewidth=1.5,label=nothing)
scatter!(ptr.solution.x[1,:],ptr.solution.x[2,:],c=:deepskyblue3,linewidth=1.5,label=nothing)
for (idx,(ce, H)) in enumerate(zip(c_list, H_list))
    label = nothing
    if idx == 1
        label = "obstacle"
    end
    plot_ellipse(p2,inv(H)*inv(H),ce,"red3",label=label)
end
annotate!(0.2, 0.5, text("Initial", :black, :center))
annotate!(4.8, 4.5, text("Final", :black, :center))
display(p2)

In [ ]:
p1 = plot(ptr.solution.t,ptr.solution.u[1,:])
plot!(ptr.solution.t,ptr.solution.t.*0 .+ vmax,linestyle=:dash)
plot!(ptr.solution.t,ptr.solution.t.*0 .- vmax,linestyle=:dash)
p2 = plot(ptr.solution.t,ptr.solution.u[2,:])
plot!(ptr.solution.t,ptr.solution.t.*0 .+ wmax,linestyle=:dash)
plot!(ptr.solution.t,ptr.solution.t.*0 .- wmax,linestyle=:dash)
plot(p1,p2,layout=(1,2),size=(500,220))

# save

In [ ]:
my_dict = Dict("x" => ptr.solution.x, "u" => ptr.solution.u, "t" => ptr.solution.t)

In [ ]:
# using JLD2, FileIO

# @save "./data/unicycle_N15_traj" my_dict